# Age prediction from eeg features

In [1]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
from utils import visualize, create_dataset_age, create_dataset_mri, cv
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import seaborn as sns
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import pandas as pd
from sklearn.linear_model import Lasso

## Create dataset

In [2]:
target = 'SWAN_Avg'
# use average and clusters eeg features
# consider all the patients
data = create_dataset_mri(SCORE = target)
# remove the diagnosis colums
data.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
data = data.rename(columns={'id':'ID'})

print(data.shape)
data.dropna(axis=0, inplace=True, subset=[target])
print(data.shape)

/Users/alestolfo/anaconda2/envs/ds-lab/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3183: DtypeWarning: Columns (50,78,80,91,92,93,94,95,105,106,107,108,109,119,120,121,123,133,134,135,137,276,291,292,295,296,297,300,301,302,305,306,307) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


(1146, 371)
(1047, 371)


In [3]:
test_indices = pd.read_csv('data/test_IDS.csv')
# Separate test and train set
data_test = pd.merge(data, test_indices, on='ID', how='inner')
data_train = data.loc[~data['ID'].isin(list((set(test_indices['ID']))))]
# labels and features
y_train = data_train[target]
X_train = data_train.drop([target, 'ID'], axis=1)
y_test = data_test[target]
X_test = data_test.drop([target, 'ID'], axis=1)

## Fix a baseline (mean age)

In [4]:
# Baseline
mean = np.mean(y_train)
print('mean_age = {}'.format(mean))
baseline_MSE = sum((mean - y_train)**2)/len(y_train)
print('baseline_MSE = {}'.format(baseline_MSE))

mean_age = 0.4441461899141631
baseline_MSE = 1.0989534552081381


In [5]:
print('Baseline test error: {}'.format(sum((mean - y_test)**2)/len(y_test)))

Baseline test error: 0.9636393545352753


# Explore different regressors and feature selection procedures

## Use AVERAGE features

## ExtraTrees + SVR

In [6]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median'],
            'regression__C' : [0.5, 0.8, 1, 2],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],
            'feat_select__threshold' : [-np.inf],
            'feat_select__max_features' : [10, 20, 50, 60]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring =  'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test R^2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 13.1min finished


Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 12.2min finished


Fitting 5 folds for each of 96 candidates, totalling 480 fits


...

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 12.3min finished



Average expected test MSE: 1.1155944280738175

Average expected test R^2: -0.020777353896919194

Average expected test mae: 0.8078557383013981
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 15.6min finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'feat_select__max_features': 50, 'feat_select__threshold': -inf, 'imputation__strategy': 'mean', 'regression__C': 0.5, 'regression__epsilon': 0.1, 'regression__kernel': 'rbf'}
True test error: 0.9545700643687751
Test r2: 0.006249807447861744
Test mae: 0.7582741234035589


# XGBoost

In [6]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__booster__alpha' : [0, 0.05, 0.1, 0.2],
              'regression__booster__max_depth' : [3, 4, 6]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring =  'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen   
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   43.4s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.9min finished


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.6min finished


Fitting 5 folds for each of 36 candidates, totalling 180 fits


...

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.6min finished



Average expected test MSE: 1.1605145235126204

Average expected test r2: -0.0724811477686003

Average expected test mae: 0.8339342304264219
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  4.5min finished


Selected hyperparameters: {'imputation__strategy': 'mean', 'regression__booster__alpha': 0, 'regression__booster__max_depth': 3}
True test error: 1.0328052911497267
Test r2: -0.07519656781568895
Test mae: 0.781719940752444


## Lasso

In [7]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('regression', Lasso())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__alpha' : [1, 1.2, 1.4, 1.7, 1.9, 2, 2.3, 2.5, 3, 3.5, 4, 6],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2=True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


...

[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



Average expected test MSE: 1.1003068880673905

Average expected test r2: -0.004502408769313471

Average expected test mae: 0.7962430920258633
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 173 out of 180 | elapsed:  1.4min remaining:    3.5s


Selected hyperparameters: {'imputation__strategy': 'mean', 'regression__alpha': 1}
True test error: 0.9636393545352754
Test r2: -0.0031917298323069776
Test mae: 0.7545917374790073


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.5min finished
